## 对所有自选股做回测

In [1]:
import pandas as pd
import yfinance as yf
import numpy as np
from statsmodels.tsa.stattools import coint
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import ParameterGrid
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import plotly.express as px

In [2]:
def get_stock_data(stock_symbols, start_date, end_date):
    stock_data = {}
    for symbol in stock_symbols:
        try:
            stock_data[symbol] = yf.download(symbol, start=start_date, end=end_date)['Adj Close']
        except Exception as e:
            print(f"Error downloading data for {symbol}: {e}")
            continue
    return pd.DataFrame(stock_data)

# 进行协整检验
def cointegration_test(y, x):
    score, p_value, _ = coint(y, x)
    return p_value

# 计算价格比率的均值和标准差，并计算均值回归的特性
def calculate_price_ratio_features(target, pair):
    price_ratio = target / pair
    mean_ratio = price_ratio.mean()
    std_ratio = price_ratio.std()
    return mean_ratio, std_ratio

# 计算欧氏距离
def calculate_euclidean_distance(target, pair):
    return np.linalg.norm(target - pair)

# 计算选择出配对股票
def pair_selection(stock_data, target_stock, cointegration_threshold):
    # 计算目标股票与潜在配对股票的相关系数
    correlations = stock_data.corr()[target_stock]
    
    # 选择相关系数大于某个阈值的股票
    correlation_threshold = 0.5
    highly_correlated_stocks = correlations[correlations > correlation_threshold].index.tolist()
    highly_correlated_stocks.remove(target_stock)
    
    cointegration_results = {}
    for stock in highly_correlated_stocks:
        p_value = cointegration_test(stock_data[target_stock], stock_data[stock])
        cointegration_results[stock] = p_value

    # 选择协整关系显著的股票
    cointegrated_stocks = [stock for stock, p_value in cointegration_results.items() if p_value < cointegration_threshold]
    
    if cointegrated_stocks == []:
        # print(f"No Pair Stock for threshold {cointegration_threshold}")
        return []

    price_ratios_mean = {}
    price_ratios_std = {}
    distances = {}

    for stock in cointegrated_stocks:
        mean_ratio, std_ratio = calculate_price_ratio_features(stock_data[target_stock], stock_data[stock])
        price_ratios_mean[stock] = mean_ratio
        price_ratios_std[stock] = std_ratio
        distances[stock] = calculate_euclidean_distance(stock_data[target_stock], stock_data[stock])
    # 转换为DataFrame
    
    price_ratios_mean_df = pd.DataFrame.from_dict(price_ratios_mean, orient='index', columns=['Mean_Ratio'])
    price_ratios_std_df = pd.DataFrame.from_dict(price_ratios_std, orient='index', columns=['Std_Ratio'])
    distances_df = pd.DataFrame.from_dict(distances, orient='index', columns=['Distance'])
    
    # 因子标准化
    scaler = StandardScaler()
    price_ratios_mean_scaled = scaler.fit_transform(price_ratios_mean_df)
    price_ratios_std_scaled = scaler.fit_transform(price_ratios_std_df)
    distances_scaled = scaler.fit_transform(distances_df)

    # 创建标准化后的DataFrame
    factor_data = pd.DataFrame({
        'Mean_Ratio_Std': price_ratios_mean_scaled.flatten(),
        'Std_Ratio_Std': price_ratios_std_scaled.flatten(),
        'Distance_Std': distances_scaled.flatten()
    }, index=cointegrated_stocks)

    # 因子加权
    factor_data['Composite_Score'] = factor_data.mean(axis=1)

    # 选择综合得分最高的股票
    pair_stock = factor_data['Composite_Score'].idxmax()
    print("Selected stock for pair trading with {}: {}".format(target_stock, pair_stock))    
    return pair_stock
    
# 定义交易信号生成函数
def generate_trading_signals(target_data, pair_data, window, threshold):
    price_ratio = target_data / pair_data
    mean_ratio = price_ratio.rolling(window=window).mean()
    std_ratio = price_ratio.rolling(window=window).std()
    
    z_score = (price_ratio - mean_ratio) / std_ratio
    
    buy_signal = z_score < -threshold
    sell_signal = z_score > threshold
    close_signal = abs(z_score) < 0.5 * threshold
    
    return buy_signal, sell_signal, close_signal

# 定义交易信号生成函数
def generate_realtime_trading_signals(target_data, pair_data, window, threshold):
    price_ratio = target_data / pair_data
    mean_ratio = price_ratio.rolling(window=window).mean()
    std_ratio = price_ratio.rolling(window=window).std()
    
    z_score = (price_ratio - mean_ratio) / std_ratio
    
    buy_signal = z_score.iloc[-1] < -threshold
    sell_signal = z_score.iloc[-1] > threshold
    close_signal = abs(z_score.iloc[-1]) < 0.5 * threshold
    
    return buy_signal, sell_signal, close_signal

def backtest(target_data, buy_signal, sell_signal, close_signal, initial_capital, transaction_cost, stop_loss, take_profit, max_position):
    capital = initial_capital
    position_target = 0  # 初始化目标股票仓位
    cash = capital
    portfolio_values = []
    positions = []  # 记录仓位变化
    buy_dates = []
    sell_dates = []
    
    for i in range(len(target_data)):
        portfolio_value = cash + position_target * target_data.iloc[i]
        # 止盈止损
        stop_loss_signal = portfolio_value < initial_capital * (1 - stop_loss)
        take_profit_signal = portfolio_value > initial_capital * (1 + take_profit)
        if buy_signal.iloc[i] and position_target == 0:
            # 考虑手续费的最大购买仓位
            max_shares = int((cash / (target_data.iloc[i] * (1 + transaction_cost))) // 100) * 100
            position_target = min(max_shares, max_position)
            cash -= position_target * target_data.iloc[i] * (1 + transaction_cost)
            buy_dates.append(target_data.index[i])
            # print(f"Buying: Target Position: {position_target}, Cash: {cash}")
        elif (sell_signal.iloc[i] or close_signal.iloc[i] or stop_loss_signal or take_profit_signal) and position_target > 0:
            cash += position_target * target_data.iloc[i] * (1 - transaction_cost)
            # print(f"Selling Target: Target Position: {position_target}, Cash: {cash}")
            position_target = 0
            sell_dates.append(target_data.index[i])
        
        portfolio_value = cash + position_target * target_data.iloc[i]
        portfolio_values.append(portfolio_value)
        positions.append(position_target)
        
    
    return portfolio_values, positions, buy_dates, sell_dates

def find_best_params(stock_data, target_stock, pair_stock, initial_capital, transaction_cost, stop_loss, take_profit, max_position):
    # 网格搜索参数优化
    param_grid = {
        'window': [10, 20, 30],
        'threshold': [1.5, 2, 2.5]
    }

    best_params = None
    best_performance = -np.inf

    for params in ParameterGrid(param_grid):
        window = params['window']
        threshold = params['threshold']
        
        # 生成交易信号
        buy_signal, sell_signal, close_signal = generate_trading_signals(stock_data[target_stock], stock_data[pair_stock], window, threshold)
        
        # 回测策略
        portfolio_values, _ , _ , _ = backtest(stock_data[target_stock], buy_signal, sell_signal, close_signal, initial_capital, transaction_cost, stop_loss, take_profit, max_position)
        
        # 计算策略表现
        performance = portfolio_values[-1] - initial_capital
        
        if performance > best_performance:
            best_performance = performance
            best_params = params

    print(f"Best Parameters: {best_params}")

    # 最佳参数生成交易信号
    window = best_params['window']
    threshold = best_params['threshold']
    return window, threshold

def visualization(stock_data, target_stock, pair_stock, buy_dates, sell_dates, portfolio_values, positions, window, threshold):
    # 可视化1：投资组合价值
    fig1 = go.Figure()
    fig1.add_trace(go.Scatter(x=stock_data.index[:len(portfolio_values)], y=portfolio_values, mode='lines', name='Portfolio Value'))
    fig1.update_layout(title='Statistical Arbitrage Backtest - Portfolio Value',
                    xaxis_title='Date',
                    yaxis_title='Portfolio Value',
                    template='plotly_dark')

    # 可视化2：目标与配对股票的股价
    fig2 = go.Figure()
    fig2.add_trace(go.Scatter(x=stock_data.index, y=stock_data[target_stock], mode='lines', name=target_stock))
    fig2.add_trace(go.Scatter(x=stock_data.index, y=stock_data[pair_stock], mode='lines', name=pair_stock))
    fig2.add_trace(go.Scatter(x=buy_dates, y=stock_data.loc[buy_dates, target_stock], mode='markers', name='Buy', marker=dict(symbol='triangle-up', size=10, color='green')))
    fig2.add_trace(go.Scatter(x=sell_dates, y=stock_data.loc[sell_dates, target_stock], mode='markers', name='Sell', marker=dict(symbol='triangle-down', size=10, color='red')))
    fig2.update_layout(title='Target and Pair Stock Prices',
                    xaxis_title='Date',
                    yaxis_title='Price',
                    template='plotly_dark')

    # 可视化3：仓位变化
    fig3 = px.area(x=stock_data.index[:len(positions)], y=positions, labels={'x': 'Date', 'y': 'Position'}, title='Position Changes', template='plotly_dark')

    # 可视化4：z_score比率
    price_ratio = stock_data[target_stock] / stock_data[pair_stock]
    mean_ratio = price_ratio.rolling(window=window).mean()
    std_ratio = price_ratio.rolling(window=window).std()
    z_score = (price_ratio - mean_ratio) / std_ratio
    fig4 = go.Figure()
    fig4.add_trace(go.Scatter(x=stock_data.index, y=z_score, mode='lines', name='z_score'))
    fig4.add_trace(go.Scatter(x=stock_data.index, y=[threshold] * len(stock_data), mode='lines', name='Upper Threshold', line=dict(dash='dash')))
    fig4.add_trace(go.Scatter(x=stock_data.index, y=[-threshold] * len(stock_data), mode='lines', name='Lower Threshold', line=dict(dash='dash')))
    fig4.update_layout(title='Z-Score of Price Ratio',
                    xaxis_title='Date',
                    yaxis_title='Z-Score',
                    template='plotly_dark')

    # 显示所有图表
    fig1.show()
    fig2.show()
    fig3.show()
    fig4.show()



In [3]:
# 参数设置
initial_capital = 10000
transaction_cost = 0.005  # 交易成本假设为0.1%
stop_loss = 0.1  # 止损10%
take_profit = 0.25  # 止盈20%
max_position = 5000  # 最大持仓

# 定义时间范围
start_date = '2024-01-01'
end_date = pd.Timestamp.today().strftime('%Y-%m-%d')

In [4]:
# 从Excel文件读取数据
df = pd.read_excel('Stocks.xlsx', sheet_name='Target-Pair')
df = df.astype(str)

# 转换为字典
stock_pairs = {row['Stock Symbol']: row['Potential Pair'].replace("'","").split(', ') for _, row in df.iterrows()}

for target_stock, potential_pairs in stock_pairs.items():
    print(f"Processing {target_stock}...")
    # 获取目标股票和潜在配对股票的数据
    all_stocks = [target_stock] + potential_pairs
    stock_data = get_stock_data(all_stocks, start_date, end_date)
    # 数据清洗，去除缺失值
    stock_data = stock_data.dropna()
    cointegration_threshold = 0.05
    pair_stock = pair_selection(stock_data, target_stock, cointegration_threshold)
    while len(pair_stock) == 0 and cointegration_threshold <= 0.5:
        cointegration_threshold += 0.05
        pair_stock = pair_selection(stock_data, target_stock, cointegration_threshold)
    if pair_stock == []:
        print(f"Cannot Find Pair Stock For {target_stock}!")
        continue
    
    print(f"Cointegration Threshold Used: {cointegration_threshold}")    
    window, threshold = find_best_params(stock_data, target_stock, pair_stock, initial_capital, transaction_cost, stop_loss, take_profit, max_position)
    buy_signal, sell_signal, close_signal = generate_trading_signals(stock_data[target_stock], stock_data[pair_stock], window, threshold)
    # 回测策略并获取结果
    portfolio_values, positions, buy_dates, sell_dates = backtest(stock_data[target_stock], buy_signal, sell_signal, close_signal, initial_capital, transaction_cost, stop_loss, take_profit, max_position)
    # 可视化
    visualization(stock_data, target_stock, pair_stock, buy_dates, sell_dates, portfolio_values, positions, window, threshold)
    

Processing 002299.SZ...


[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


Selected stock for pair trading with 002299.SZ: 603477.SS
Cointegration Threshold Used: 0.2
Best Parameters: {'threshold': 2, 'window': 10}


[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed

Processing 600938.SS...


...


[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


Selected stock for pair trading with 601985.SS: 003816.SZ
Cointegration Threshold Used: 0.1
Best Parameters: {'threshold': 1.5, 'window': 20}


Processing 600900.SS...


[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


Selected stock for pair trading with 600900.SS: 600025.SS
Cointegration Threshold Used: 0.25
Best Parameters: {'threshold': 1.5, 'window': 30}
